In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import requests
from datasets import load_from_disk


In [14]:
tokenizer = AutoTokenizer.from_pretrained("/root/group-shared/jrc/base_models/Qwen3-14B")
model = AutoModelForCausalLM.from_pretrained(
    "/root/group-shared/jrc/base_models/Qwen3-14B",
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [3]:
dataset = load_from_disk("/root/workspace/PRewrite/data/aime_2024_only")
meta_instruction = "Rewrite the following instruction via rephrasing and/or adding specific requirements. Add instructions which would be helpful to solve the problem correctly. Output the new instruction only."

def add_meta_instruction(element):
    tokenizer = AutoTokenizer.from_pretrained("/root/group-shared/jrc/base_models/Qwen3-14B")
    element["prompt"] = tokenizer.apply_chat_template(
        [
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": f"{meta_instruction}\nInstruction: {element['instruction']}"
            }
        ],
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False,
    )
    return element

dataset = dataset.map(add_meta_instruction, num_proc = 8, desc = "Adding meta instruction")

print(dataset["train"][0])

num_proc must be <= 1. Reducing num_proc to 1 for dataset of size 1.
num_proc must be <= 1. Reducing num_proc to 1 for dataset of size 1.


{'instruction': 'Provide a step-by-step solution to the hard math problem, with the final answer boxed at the end.', 'dataset_name': 'aime_2024', 'prompt': '<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nRewrite the following instruction via rephrasing and/or adding specific requirements. Add instructions which would be helpful to solve the problem correctly. Output the new instruction only.\nInstruction: Provide a step-by-step solution to the hard math problem, with the final answer boxed at the end.<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\n'}


In [5]:
def change_initial_instruction(element):
    tokenizer = AutoTokenizer.from_pretrained("/root/group-shared/jrc/base_models/Qwen3-14B")
    inputs = tokenizer(
        [element["prompt"]],
        return_tensors="pt"
    )
    
    inputs.to(model.device)

    output_ids = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=1024,
        do_sample=False
    )
    element["instruction"] = tokenizer.decode(output_ids[0], skip_special_tokens=True).split("</think>")[-1].strip()
    element["prompt"] = tokenizer.apply_chat_template(
        [
            {
                "role": "system",
                "content": "You are a helpful assistant."
            },
            {
                "role": "user",
                "content": f"{meta_instruction}\nInstruction: {element['instruction']}"
            }
        ],
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False,
    )
    return element

dataset = dataset.map(change_initial_instruction, desc = "Changing initial instruction")

print(dataset["train"][0])

Changing initial instruction:   0%|          | 0/1 [00:00<?, ? examples/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Changing initial instruction:   0%|          | 0/1 [00:00<?, ? examples/s]

{'instruction': 'Provide a clear, step-by-step solution to the challenging math problem, explaining each part of the process in detail. Ensure that all mathematical operations are shown correctly, and include any necessary formulas or reasoning. Make sure the solution is complete and leads logically to the final answer, which should be clearly presented in a box at the end.', 'dataset_name': 'aime_2024', 'prompt': '<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nRewrite the following instruction via rephrasing and/or adding specific requirements. Add instructions which would be helpful to solve the problem correctly. Output the new instruction only.\nInstruction: Provide a clear, step-by-step solution to the challenging math problem, explaining each part of the process in detail. Ensure that all mathematical operations are shown correctly, and include any necessary formulas or reasoning. Make sure the solution is complete and leads logically to the final a

In [9]:
meta_prompt = """Rewrite the following via rephrasing and/or adding specific requirements. According to the subject of the problem, add particular instructions which would be helpful to solve the problem correctly. Output the new instruction only.
Instruction: Solution to the math problem in subject **Number Theory**."""

prompt = tokenizer.apply_chat_template(
    [
        {
            "role": "system",
            "content": "You are a helpful assistant."
        },
        {
            "role": "user",
            "content": meta_prompt
        }
    ],
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False,
)
print(prompt)

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
Rewrite the following via rephrasing and/or adding specific requirements. According to the subject of the problem, add particular instructions which would be helpful to solve the problem correctly. Output the new instruction only.
Instruction: Solution to the math problem in subject **Number Theory**.<|im_end|>
<|im_start|>assistant
<think>

</think>




In [10]:
generate_model_config = {
    "temperature": 1,
    "max_new_tokens": 4096,
}
request = requests.post(
    "http://0.0.0.0:33337/generate",
    json={"text": prompt, "sampling_params": generate_model_config},
)
rewrite_prompt = request.json()['text']
print(rewrite_prompt)

Provide a detailed solution to the math problem in the subject **Number Theory**, including step-by-step reasoning. Use appropriate theorems, properties, or lemmas relevant to number theory (such as divisibility rules, modular arithmetic, prime factorization, or the fundamental theorem of arithmetic) as needed. Ensure clarity in your logic and explicitly show how each step contributes to the final solution. If the problem involves proof, write the proof in a formal and structured manner.


In [29]:
import re
print(rewrite_prompt)
tags = ["{problem}", "{subject}", "{difficulty}","{hello}"]
print(set(re.findall(r"{[^}]+}", rewrite_prompt)) == set(tags))

# Instruction
Provide a detailed and step-by-step solution to the challenging math problem: {problem}  
The subject of the problem is {subject}.  
The level of difficulty is {difficulty}.  
Ensure your solution includes clear explanations for each step, any relevant formulas or theorems used, and, where applicable, a logical breakdown of the problem-solving process. Verify your final answer to ensure correctness.
False


In [25]:
data = {
    "problem": "Wendy is five times as old as Colin will be seven years from now. In 25 years, Colin will be a third as old as Wendy is now. Compute Colin's current age.",
    "subject": "math",
    "difficulty": 1,
    "hello": "world"
}
print(rewrite_prompt.format(**data))

# Instruction
Provide a detailed and step-by-step solution to the challenging math problem: Wendy is five times as old as Colin will be seven years from now. In 25 years, Colin will be a third as old as Wendy is now. Compute Colin's current age.  
The subject of the problem is math.  
The level of difficulty is 1.  
Ensure your solution includes clear explanations for each step, any relevant formulas or theorems used, and, where applicable, a logical breakdown of the problem-solving process. Verify your final answer to ensure correctness.


In [ ]:
# 方法1: 使用清理函数（推荐）
# 注意：如果clear_gpu_memory函数还没定义，先运行Cell 10
try:
    clear_gpu_memory(
        model=model,
        functions_to_clear=['change_initial_instruction']  # 删除可能持有模型引用的函数
    )
except NameError:
    # 如果函数还没定义，使用手动清理
    print("⚠️ clear_gpu_memory函数未定义，使用手动清理...")
    import torch
    import gc
    
    # 删除函数中的闭包引用
    if 'change_initial_instruction' in globals():
        del change_initial_instruction
    
    # 删除模型
    if 'model' in globals():
        del model
    
    # 清理GPU
    if torch.cuda.is_available():
        for i in range(torch.cuda.device_count()):
            with torch.cuda.device(i):
                torch.cuda.empty_cache()
        torch.cuda.synchronize()
    
    gc.collect()
    print("✅ 手动清理完成")

In [ ]:
# 彻底清理显存的方法
import torch
import gc

# 方法1: 删除所有对模型的引用
# 注意：在notebook中，需要确保没有其他变量引用模型
try:
    del model
except NameError:
    pass

# 方法2: 清理所有GPU缓存
if torch.cuda.is_available():
    # 清理所有GPU设备
    for i in range(torch.cuda.device_count()):
        with torch.cuda.device(i):
            torch.cuda.empty_cache()
            torch.cuda.ipc_collect()  # 清理进程间通信缓存
    
    # 同步所有设备
    torch.cuda.synchronize()

# 方法3: Python垃圾回收
gc.collect()

# 方法4: 强制清理（如果使用accelerate的device_map）
try:
    import accelerate
    # 如果模型使用了accelerate的设备映射，需要额外清理
    pass
except ImportError:
    pass

# 检查显存使用情况
if torch.cuda.is_available():
    print("GPU显存使用情况:")
    for i in range(torch.cuda.device_count()):
        allocated = torch.cuda.memory_allocated(i) / 1024**3  # GB
        reserved = torch.cuda.memory_reserved(i) / 1024**3  # GB
        print(f"  GPU {i}: 已分配 {allocated:.2f} GB, 已保留 {reserved:.2f} GB")
else:
    print("CUDA不可用")

15604

In [ ]:
# 通用的显存清理函数（推荐使用）
def clear_gpu_memory(model=None, tokenizer=None, functions_to_clear=None):
    """
    彻底清理GPU显存
    
    Args:
        model: 要删除的模型对象
        tokenizer: 要删除的tokenizer对象（可选）
        functions_to_clear: 要删除的函数列表，这些函数可能持有模型的闭包引用
    """
    import torch
    import gc
    
    # 1. 删除可能持有模型引用的函数
    if functions_to_clear:
        for func_name in functions_to_clear:
            if func_name in globals():
                del globals()[func_name]
                print(f"✅ 已删除函数: {func_name}")
    
    # 2. 删除模型
    if model is not None:
        # 如果模型使用了device_map，需要特殊处理
        if hasattr(model, 'hf_device_map') and model.hf_device_map:
            # 对于使用device_map的模型，需要手动清理每个设备
            for device in model.hf_device_map.values():
                if isinstance(device, str) and 'cuda' in device:
                    device_id = int(device.split(':')[-1]) if ':' in device else 0
                    with torch.cuda.device(device_id):
                        torch.cuda.empty_cache()
        
        # 删除模型对象
        del model
        print("✅ 模型已删除")
    
    # 3. 删除tokenizer（可选）
    if tokenizer is not None:
        del tokenizer
        print("✅ Tokenizer已删除")
    
    # 4. 清理所有GPU缓存
    if torch.cuda.is_available():
        for i in range(torch.cuda.device_count()):
            with torch.cuda.device(i):
                torch.cuda.empty_cache()
                torch.cuda.ipc_collect()
        torch.cuda.synchronize()
        print("✅ GPU缓存已清理")
    
    # 5. Python垃圾回收
    gc.collect()
    print("✅ Python垃圾回收完成")
    
    # 6. 显示显存使用情况
    if torch.cuda.is_available():
        print("\n📊 当前GPU显存使用情况:")
        for i in range(torch.cuda.device_count()):
            allocated = torch.cuda.memory_allocated(i) / 1024**3
            reserved = torch.cuda.memory_reserved(i) / 1024**3
            print(f"  GPU {i}: 已分配 {allocated:.2f} GB, 已保留 {reserved:.2f} GB")

# 使用示例：
# clear_gpu_memory(
#     model=model,
#     tokenizer=tokenizer,  # 可选
#     functions_to_clear=['change_initial_instruction']  # 可选
# )


In [15]:
del model

In [18]:
import torch
import gc
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        with torch.cuda.device(i):
            torch.cuda.empty_cache()
            torch.cuda.ipc_collect()
    torch.cuda.synchronize()
    print("✅ GPU缓存已清理")
gc.collect()

✅ GPU缓存已清理


0

In [17]:
gc.collect()

0

In [ ]:
del trainer.model
if hasattr(trainer, 'ref_model') and trainer.ref_model is not None:
    del trainer.ref_model
del trainer

# 清空显存
if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()